In [74]:
import pandas as pd
df_2021 = pd.read_csv("2021.csv");
df_2020 = pd.read_csv("2020.csv");
df_2019 = pd.read_csv("2019.csv");
df_2018 = pd.read_csv("2018.csv");

df = pd.concat([df_2018, df_2019, df_2020, df_2021], ignore_index=True);
ignore_tx_types = ["Fiat Deposit", "Fiat Withdrawal"]
df = df[(~df["Trade Type"].isin(ignore_tx_types))]
ignore_currencies = ["Canadian Dollar (CAD)", "YTUSD", "Balancer Pool Token (BPT)"]
df = df[(~df.Currency.isin(ignore_currencies))]
df = df[(df.From != "Binance")]
#ignore all binance transactions



df.Timestamp = pd.to_datetime(df.Timestamp)
df.Quantity = df.Quantity.str.replace(",", "")
df.Quantity = pd.to_numeric(df.Quantity)
df["Trade Type"].unique()
#df = df[df.Currency == "Cardano (ADA)"]
df.tail()

,Currency,Timestamp,Trade Type,Price,Quantity,Value (Incl. Fee),Fee,From,To,Transaction Id
10914,(PoS) Dai Stablecoin (DAI),2021-12-30 00:13:00,Staking Deposit,1.281953,2601.090000,3334.47,0.000000,Joint Polygon (0x8A...bf0C),0x27f8d03b3a2196956ed754badc28d73be8830a6e,0xa1cd48ded0b78b5395c750c99a98d48d440c4386bda3...
10915,Polygon (MATIC),2021-12-30 00:13:00,Fee,3.123383,0.006412,0.02,0.000000,Joint Polygon (0x8A...bf0C),Joint Polygon (0x8A...bf0C),0xa1cd48ded0b78b5395c750c99a98d48d440c4386bda3...
10916,Polygon (MATIC),2021-12-30 00:14:00,Fee,3.129772,0.004324,0.01,0.000000,Joint Polygon (0x8A...bf0C),Joint Polygon (0x8A...bf0C),0xeefdad4a3e46e40f4d074fdeb95403a1b1e155fd2eb1...
10917,Wrapped Matic (WMATIC),2021-12-30 16:37:00,Staking,3.169829,4.444075,14.09,0.000673,0x2bb25175d9b0f8965780209eb558cc3b56ca6d32,Wallace Polygon (0xFC...49C3),0x5c17ddd840d8a859e07cac072c6f3930a20b15207188...
10918,Polygon (MATIC),2021-12-30 16:37:00,Fee,3.298466,0.000204,0.00,0.000000,Wallace Polygon (0xFC...49C3),Wallace Polygon (0xFC...49C3),0x5c17ddd840d8a859e07cac072c6f3930a20b15207188...


In [75]:
# numeric_columns = ["Quantity", "Opening Cost", "Opening Value", "Ending Cost", "Closing Value", "Opening Balance", "Closing Balance"]

# for x in numeric_columns:
#     df[x] = pd.to_numeric(df[x].str.replace(",","").replace("^-$", "0", regex=True))


In [76]:
#get all trade types
df["Trade Type"].unique()
income_types = ["Income", "Airdrop"]
income_transactions = df[df["Trade Type"].isin(income_types) ]

tax_types = ["Income", "Airdrop", "Mining", "Cashback", "Borrow", "Unknown Acquisition", "Transfer In", "Staking Withdrawal"]
#remove all income transactions from tax calculations
tax_df = df[~df["Trade Type"].isin(income_types)]


income_types = ["Income", "Airdrop"]
income_tx = df[df["Trade Type"].isin(income_types) ]

non_taxed_types = ["Cashback", "Borrow", "Unknown Acquisition", "Mining"]
non_tax_df = df[df["Trade Type"].isin(non_taxed_types)]

staking_types =["Staking", "Staking Withdrawal"]
staked_df = df[df["Trade Type"].isin(staking_types)]

In [77]:
# store all income tokens seperately
all_coins = df.Currency.unique()
income_tokens = {}
for token_name in all_coins:
    this_income_tokens = tax_df[tax_df.Currency == token_name ]
    income_tokens[token_name] = this_income_tokens

In [78]:
#create a column to store the ACB at each sell transactions
tax_df["ACB"]  = None;
#tax_df.apply(lambda x: print (x.name), axis=1)
taxable_events_df = pd.DataFrame(columns = ["Timestamp","Currency", "Quantity", "Price", "TradeType", "Gains", "Description", "Warnings"])


def calc_acb(prev_acb, row):
    result = {"cost": 0,  "proceed": 0, "fee": 0}

    #read the last acb
    if (prev_acb is not None):
        this_acb = prev_acb.copy() 
    else:
        this_acb = pd.DataFrame(columns = ["Currency", "Quantity", "Price", "Trade Type", "Timestamp"])
        row["ACB"] = this_acb
#tax_types = ["Income", "Airdrop", "Mining", "Cashback", "Borrow", "Unknown Acquisition", "Transfer In"]
 
    trade_type = row["Trade Type"]

    if (trade_type in ["Burn", "Sell", "Fee", "Staking", "Staking Deposit", "Failed Out", "Loan Repayment"] ):
        
        #selling, 
        sell_quantity = row["Quantity"]
        left_over_sell_quantity = sell_quantity
        rows_to_delete = [];
        curr_cost = 0;
        tax_desc = []
        isNegBalance = False
        for _, acb_row in this_acb.iterrows():
            #for each row, deduct the amount to be sold 

            delta = min(left_over_sell_quantity, acb_row.Quantity)
            left_over_sell_quantity -= delta
            acb_type = acb_row["Trade Type"]
            acb_timestamp = acb_row.Timestamp
            acb_quantity = delta
            acb_currency = acb_row.Currency
            acb_cost = 0
            if (acb_type in ["Staking Withdrawal", "Mint", "Income", "Airdrop", "Mining", "Cashback", "Borrow", "Unknown Acquisition", "Transfer In"]):
                # these types of incoming tokens will not be counted for any cost , 
                # so use today's price to calc cost so that no gain can incur
                acb_cost = row.Price * delta
                curr_cost += acb_cost
            else:
                #otherwise the cost of the acb entry will be the actual cost
                acb_cost = acb_row.Price * delta
                curr_cost += acb_cost

            tax_desc.append(f">>> from {acb_type} on {acb_timestamp} for {acb_quantity} {acb_currency} worth ${acb_cost}")

            if (acb_row.Quantity - delta < 0) :
                rows_to_delete.append(acb_row.name)
            else:
                this_acb.at[acb_row.name, "Quantity"] -= delta

            if (left_over_sell_quantity == 0):
                #if there are more to sell remove this row
                break;

        #if there is a negative balance, then use today's price to calculate the cost basis
        isNegBalance = left_over_sell_quantity > 0
        curr_cost += left_over_sell_quantity * row.Price
       
        if (isNegBalance):
            tax_desc.append(">>> WARNING: -ve balance:" + str(left_over_sell_quantity))

        result["cost"] = curr_cost
        result["proceed"] = sell_quantity * row.Price

        gains = result["proceed"] - result["cost"]
        if (trade_type in ["Fees"]):
            fee = sell_quantity * row.Price
            result["fee"] = fee
            print(f"Paid Fee {row.Quantity} {row.Currency} @ {row.Price} on {row.Timestamp} fee ${fee} gained {gains}");
        elif (trade_type in ["Staking Deposit", "Staking"]):
            gains = 0
            result["cost"] = 0
            result["proceed"] = 0
            print(f"Staked {row.Quantity} {row.Currency} on {row.Timestamp}");
        else:
            print(f"Selling {row.Quantity} {row.Currency} @ {row.Price} on {row.Timestamp} gained {gains}");

         
        tax_desc = "\n".join(tax_desc)
        print(tax_desc)
# taxable_events_df = pd.DataFrame(columns = ["Currency", "Quantity", "Price", "TradeType", "Description"])

        taxable_events_df.loc[len(taxable_events_df.index)] = [row.Timestamp, row.Currency, row.Quantity, row.Price, trade_type, gains, tax_desc, "-ve Balance" if isNegBalance else None]
        this_acb.drop(index = rows_to_delete , inplace=True)
            #otherwise go on to the next row
    elif (trade_type in ["Mint", "Staking Withdrawal", "Interest", "Income", "Airdrop", "Mining", "Cashback", "Borrow", "Unknown Acquisition", "Transfer In", "Buy"]):
        #add to the acb
        print(f"{trade_type} {row.Quantity} {row.Currency} @ {row.Price} on {row.Timestamp}");
        this_acb.loc[len(this_acb.index)] = [row.Currency, row.Quantity, row.Price, trade_type, row.Timestamp]
    elif(trade_type in ["Transfer Out"]):
        print(f"{trade_type} {row.Quantity} {row.Currency} @ {row.Price} on {row.Timestamp}");
    else:
        print(f"{trade_type} not yet recognized")
        raise f"{trade_type} not yet recognized"

    return this_acb[this_acb["Quantity"] > 0]

#go through all the tokens, calculate the ACB
all_coins = df.Currency.unique()

for token_name in all_coins:
    token_txs = tax_df[(tax_df.Currency == token_name) ]
    curr_acb = None
    for index, row in token_txs.iterrows():
        curr_acb = calc_acb(curr_acb, row)

Buy 0.0028606 Bitcoin (BTC) @ 17347.6081 on 2018-01-01 01:46:00
Selling 0.00181653 Bitcoin (BTC) @ 20433.41386 on 2018-01-06 03:42:00 gained 5.6054587372128
>>> from Buy on 2018-01-01 01:46:00 for 0.00181653 Bitcoin (BTC) worth $31.512450541893003
Buy 0.00251394 Bitcoin (BTC) @ 18046.52882 on 2018-01-10 11:15:00
Selling 0.00271777 Bitcoin (BTC) @ 14757.94851 on 2018-01-16 10:50:00 gained -8.2078827529783
>>> from Buy on 2018-01-01 01:46:00 for 0.00104407 Bitcoin (BTC) worth $18.112117188967
>>> from Buy on 2018-01-10 11:15:00 for 0.0016737 Bitcoin (BTC) worth $30.204475286034
Selling 0.00486212 Bitcoin (BTC) @ 14782.79341 on 2018-01-16 10:51:00 gained -2.7423210408984033
>>> from Buy on 2018-01-10 11:15:00 for 0.0008402400000000001 Bitcoin (BTC) worth $15.163415375716802
>>> WARNING: -ve balance:0.00402188
Buy 0.00653363 Bitcoin (BTC) @ 14582.91287 on 2018-01-18 10:58:00
Buy 0.00243304 Bitcoin (BTC) @ 14132.2314 on 2018-01-19 04:54:00
Selling 0.00586462 Bitcoin (BTC) @ 15789.04095 on 2

C:\Users\naaht\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Mining 0.00134396 Bitcoin (BTC) @ 10561.27241 on 2018-04-20 05:03:00
Buy 0.00120139 Bitcoin (BTC) @ 11986.94457 on 2018-04-24 12:08:00
Mining 0.002569 Bitcoin (BTC) @ 11337.13833 on 2018-04-25 20:02:00
Selling 0.0012014 Bitcoin (BTC) @ 11378.45685 on 2018-05-10 20:42:00 gained 0.0
>>> from Mining on 2018-04-25 20:02:00 for 0.0012014 Bitcoin (BTC) worth $13.67007805959
Transfer In 0.004 Bitcoin (BTC) @ 11442.39362 on 2018-05-10 21:35:00
Mining 0.0015774 Bitcoin (BTC) @ 10759.67266 on 2018-05-11 19:38:00
Selling 0.00399998 Bitcoin (BTC) @ 10676.21067 on 2018-05-12 01:32:00 gained 0.0
>>> from Mining on 2018-05-11 19:38:00 for 0.0015774 Bitcoin (BTC) worth $16.840654710858
>>> WARNING: -ve balance:0.00242258
Transfer In 0.0041628 Bitcoin (BTC) @ 10966.42064 on 2018-05-15 12:54:00
Mining 0.00220389 Bitcoin (BTC) @ 9673.563678 on 2018-05-25 07:31:00
Mining 0.002577 Bitcoin (BTC) @ 9591.141112 on 2018-05-29 20:23:00
Mining 0.00102872 Bitcoin (BTC) @ 9770.841652 on 2018-06-01 05:37:00
Mining 

In [81]:
start_date = "2021-01-01"
end_date = "2021-12-31"
mask = (taxable_events_df.Timestamp >= start_date) & (taxable_events_df.Timestamp <= end_date)
taxable_events_2021 = taxable_events_df.loc[mask]

#taxable_events_df[taxable_events_df.Timestamp ]
taxable_events_2021 = taxable_events_2021.sort_values("Gains", ascending=False)
#raven = taxable_events_2021[taxable_events_2021.Currency == "Ravencoin (RVN)"]



In [80]:

#taxable_events_2021.to_csv("2021gains.csv")
taxable_events_2021.Gains.sum()

-1143.2476078614109